Long Short-Term Memory network is a type of Recurrent Neural Network

In [5]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using Theano backend.


In [6]:
# fix random seed for reproducibility
numpy.random.seed(7)


In [7]:
# load the dataset
dataframe = pandas.read_csv('tweetvolumes.csv', usecols=[1], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')

IOError: [Errno 2] No such file or directory: 'tweetvolumes.csv'

In [10]:
dataframe.describe()

,Number of Tweets
count,171.000000
mean,1204.011696
std,625.290060
min,275.000000
25%,721.500000
50%,968.000000
75%,1641.500000
max,4278.000000


In [11]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [12]:
# split into train and test sets - train set ends on May 12
train_size = 61
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size-10:len(dataset),:]
print(len(train), len(test))

(61, 120)


In [13]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

In [14]:
# reshape into X=t and Y=t+1
look_back = 7
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [15]:
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [16]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_dim=look_back))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, nb_epoch=100, batch_size=1, verbose=2)

Epoch 1/100
1s - loss: 0.1558
Epoch 2/100
1s - loss: 0.0597
Epoch 3/100
1s - loss: 0.0220
Epoch 4/100
1s - loss: 0.0099
Epoch 5/100
1s - loss: 0.0070
Epoch 6/100
1s - loss: 0.0065
Epoch 7/100
1s - loss: 0.0064
Epoch 8/100
1s - loss: 0.0065
Epoch 9/100
1s - loss: 0.0064
Epoch 10/100
1s - loss: 0.0064
Epoch 11/100
1s - loss: 0.0065
Epoch 12/100
1s - loss: 0.0064
Epoch 13/100
1s - loss: 0.0065
Epoch 14/100
1s - loss: 0.0065
Epoch 15/100
1s - loss: 0.0065
Epoch 16/100
1s - loss: 0.0063
Epoch 17/100
1s - loss: 0.0063
Epoch 18/100
1s - loss: 0.0064
Epoch 19/100
1s - loss: 0.0063
Epoch 20/100
1s - loss: 0.0063
Epoch 21/100
1s - loss: 0.0063
Epoch 22/100
1s - loss: 0.0063
Epoch 23/100
1s - loss: 0.0062
Epoch 24/100
1s - loss: 0.0063
Epoch 25/100
1s - loss: 0.0062
Epoch 26/100
1s - loss: 0.0063
Epoch 27/100
1s - loss: 0.0062
Epoch 28/100
1s - loss: 0.0062
Epoch 29/100
1s - loss: 0.0062
Epoch 30/100
1s - loss: 0.0063
Epoch 31/100
1s - loss: 0.0062
Epoch 32/100
1s - loss: 0.0061
Epoch 33/100
1s -

In [17]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

Train Score: 296.41 RMSE
Test Score: 509.65 RMSE


In [20]:
testPredict

array([[ 1981.43615723],
       [ 1992.1418457 ],
       [ 2100.40600586],
       [ 2168.33374023],
       [ 2473.52172852],
       [ 2310.02392578],
       [ 2464.37451172],
       [ 1949.58654785],
       [ 1885.75598145],
       [ 1664.08544922],
       [ 1750.41748047],
       [ 1725.79052734],
       [ 1537.6270752 ],
       [ 1409.41516113],
       [ 1293.01904297],
       [ 1386.03112793],
       [ 1284.80944824],
       [ 1394.8918457 ],
       [ 1273.22619629],
       [ 1293.39733887],
       [ 1211.71679688],
       [ 1399.80554199],
       [ 1239.98327637],
       [ 1392.62475586],
       [ 1276.40551758],
       [ 1436.38439941],
       [ 1433.35852051],
       [ 1471.65856934],
       [ 1395.31591797],
       [ 1387.54345703],
       [ 1367.40905762],
       [ 1359.68640137],
       [ 1272.97888184],
       [ 1320.84277344],
       [ 1278.33850098],
       [ 1317.51367188],
       [ 1216.58496094],
       [ 1240.67443848],
       [ 1225.46899414],
       [ 1263.38220215],


In [18]:
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict-10)+(look_back)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

ValueError: could not broadcast input array from shape (112,1) into shape (109,1)

lookback = 1, hidden neurons = 4
Train Score: 398.36 RMSE
Test Score: 314.96 RMSE

lookback = 3, hidden neurons = 4
Train Score: 387.69 RMSE
Test Score: 214.87 RMSE

lookback = 7, hidden neurons = 4
Train Score: 357.99 RMSE
Test Score: 146.85 RMSE

lookback = 10, hidden neurons = 8
Train Score: 366.99 RMSE
Test Score: 149.24 RMSE

